In [90]:
import torch
import torch.nn as nn

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from joblib import load, dump
from skorch import NeuralNetClassifier



# Przygotowanie danych do uczenia


In [91]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

data = data.drop(columns=["user_id", "track_id"])

# TODO delete?
data = data.drop(
    columns=["release_date", "key", "loudness", "popularity", "duration_ms", "explicit",
             "danceability", "energy", "speechiness",
             "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

# data = data.drop(columns=["release_date", "explicit", "key", "loudness", "favourite_genres", "genres"])


# TODO map genres

# def map_genres(genre):
#   if "rock" in genre:
#     return "rock"
#   elif "pop" in genre:
#     return "pop"
#   elif "dance" in genre:
#     return "dance"
#   elif "wave" in genre:
#     return "wave"
#   elif "metal" in genre:
#     return "metal"
#   else:
#     return genre
#
#
# data["genres"] = data["genres"].apply(lambda genres: list(set(map(map_genres, genres))))
# data["favourite_genres"] = data["favourite_genres"].apply(lambda genres: list(set(map(map_genres, genres))))

data.head(500)

,favourite_genres,genres,skipped
0,"[permanent wave, mandopop, funk]","[album rock, art rock, classic rock, folk rock...",False
1,"[filmi, regional mexican, folk]","[album rock, art rock, classic rock, folk rock...",False
2,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False
3,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False
4,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False
...,...,...,...
495,"[hard rock, alternative metal, singer-songwriter]","[album rock, art rock, blues, blues rock, brit...",False
496,"[permanent wave, mandopop, funk]","[album rock, art rock, classic rock, classic u...",True
497,"[permanent wave, mandopop, funk]","[album rock, art rock, classic rock, classic u...",True
498,"[filmi, regional mexican, folk]","[album rock, art rock, classic rock, classic u...",False


# Zamiana wartości kategorycznych (genres) na liczbowe

In [92]:
unique_genres = set.union(*data["genres"].apply(set).tolist(),
                          *data["favourite_genres"].apply(set).tolist())

encoder = LabelEncoder()
encoder.fit(list(unique_genres))

# convert the categorical values into numeric - favourite_genres is a list of strings
data["genres"] = data["genres"].apply(lambda genres: encoder.transform(genres).tolist())
data["favourite_genres"] = data["favourite_genres"].apply(
    lambda genres: encoder.transform(genres).tolist())

# convert encoded lists into binary arrays
mlb = MultiLabelBinarizer()
genres_binarized = mlb.fit_transform(data["genres"])
favourite_genres_binarized = mlb.transform(data["favourite_genres"])

# Combine the binary arrays with appropriate column suffixes
X_genres_df = pd.DataFrame(genres_binarized, columns=[f'g_{col}' for col in mlb.classes_.tolist()])
X_fav_genres_df = pd.DataFrame(favourite_genres_binarized,
                               columns=[f'fav_{col}' for col in mlb.classes_.tolist()])

# Join the binary arrays with the original DataFrame
data = data.join(X_genres_df).join(X_fav_genres_df)

# Drop the original columns
data = data.drop(columns=["genres", "favourite_genres"])

data.head(5)


KeyboardInterrupt: 

# Ekstrakcja labeli

In [ ]:
X = data.drop(columns=["skipped"])
Y = data["skipped"]

# Podział danych na zbiór treningowy i testowy

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("{}".format(Y_train.value_counts()))
print("{}".format(Y_test.value_counts()))

# Train (RandomForestClassifier)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, Y_train)

print("TRENINGOWY")
y_pred = model.predict(X_train)
print("Accuracy:", accuracy_score(Y_train, y_pred))
print("Confusion matrix:\n", confusion_matrix(Y_train, y_pred))
print("Classification report:\n", classification_report(Y_train, y_pred))

print("TESTOWY")
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(Y_test, y_pred))
print("Classification report:\n", classification_report(Y_test, y_pred))

# Train (MLPClassifier)

In [ ]:
class ClassifierModule(nn.Module):
  def __init__(self, num_inputs=3532, num_hidden=10, num_outputs=1):
    super(ClassifierModule, self).__init__()

    self.layer1 = nn.Linear(num_inputs, num_hidden)
    self.relu = nn.ReLU()
    self.layer2 = nn.Linear(num_hidden, num_outputs)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.layer1(x)
    x = self.relu(x)
    x = self.layer2(x)
    x = self.sigmoid(x)
    return x

net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    optimizer__momentum=0.9,
    verbose=0,
    train_split=False,
)

params = {
  'lr': [0.05, 0.1],
  'module__num_hidden': [1, 3],
  'optimizer__nesterov': [False, True],
}

mlpc_grid = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=2)

# mlpc_grid = GridSearchCV(MLPClassifier(), {
#   'hidden_layer_sizes': [(3), (3, 3)],
#   'activation': ['relu'],
#   'solver': ['adam'],
#   'learning_rate': ['constant'],
# }, n_jobs=-1, cv=ms.KFold(shuffle=True), verbose=10)
mlpc_grid.fit(X, Y)
df = pd.DataFrame(mlpc_grid.cv_results_)
df.drop(
  columns=["split0_test_score", "split1_test_score", "split2_test_score", "split3_test_score"])

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6518       0.6333        0.6548  1.6509
      2        0.6333       0.6333        0.6514  0.8646
      3        0.6124       0.6368        0.6506  0.8676
      4        0.5940       0.6120        0.6526  1.0697
      5        0.5795       0.5953        0.6552  0.9031
      6        0.5707       0.5812        0.6569  0.8920
      7        0.5629       0.5781        0.6580  0.7455
      8        0.5582       0.5812        0.6589  0.7556
      9        0.5531       0.5857        0.6588  0.7397
     10        0.5484       0.5827        0.6601  0.7368
     11        0.5440       0.5832        0.6599  0.9628
     12        0.5400       0.5888        0.6610  0.7439
     13        0.5356       0.5847        0.6623  0.7567
     14        0.5329       0.5867        0.6611  0.7361
     15        0.5286       0.5847        0.6626  0.7433
     16        0.5248       0.5

In [ ]:
print("TRENINGOWY")
y_pred = mlpc_grid.predict(X_train)
print("Accuracy:", accuracy_score(Y_train, y_pred))
print("Confusion matrix:\n", confusion_matrix(Y_train, y_pred))
print("Classification report:\n", classification_report(Y_train, y_pred))

print("TESTOWY")
y_pred = mlpc_grid.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(Y_test, y_pred))
print("Classification report:\n", classification_report(Y_test, y_pred))
